<a href="https://colab.research.google.com/github/ruedas91/FinTechruedas/blob/main/GC_model_persistence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Persistence

In [1]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd

In [2]:
# Upload winequality.csv to Colab
from google.colab import files

csv_file = files.upload()

Saving winequality.csv to winequality.csv


In [3]:
# Read in data
df = pd.read_csv("winequality.csv", delimiter=";")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Prepare the data

In [4]:
# Define the features X set and the target y vector
X = df.iloc[:, 0:11].values
y = df["quality"].values

In [5]:
# Scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

## Create the shallow neural network model

In [6]:
# Define the model - shallow neural network
nn = Sequential()

# Hidden layer
nn.add(Dense(units=8, input_dim=11, activation="relu"))

# Output layer
nn.add(Dense(units=1, activation="linear"))

In [7]:
# Compile the model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model = nn.fit(X, y, epochs=200, verbose=0)

## Save the model as JSON to a file

In [8]:
# Create file path for model as JSON
file_path = "model.json"

# Save model as JSON
nn_json = nn.to_json()

file_path = "model.json"
with open(file_path, "w") as json_file:
  json_file.write(nn_json)

## Save the model weights

In [9]:
# Create file path for weights
file_path = "model.h5"

# Save weights
nn.save_weights(file_path)

## Load the model and weights

In [11]:
# Load the saved model to make predictions

# Imports
from tensorflow.keras.models import model_from_json

# Create file path for model as JSON
file_path = "model.json"

# load json and create model
file_path = "model.json"
with open(file_path, "r")as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)


# Create file path for model
file_path = "model.h5"

# load weights into new model
file_path = "model.h5"
loaded_model.load_weights(file_path)

## Make predictions with the model

In [12]:
# Make some predictions with the loaded model
df["predicted"] = loaded_model.predict(X)
df.head(10)

50/50 [==============================] - 0s 3ms/step


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,predicted
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,5.012122
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,4.951518
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,5.357452
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,5.493874
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,5.012122
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,5.115373
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,5.145945
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,5.181086
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,5.407394
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,5.589226
